# AHRS

This Notebook showcases the most important classes and functions included in the Python package `ahrs`.

Here we will explore the basic use of:

- Class [DCM](https://ahrs.readthedocs.io/en/latest/dcm/classDCM.html)
- Class [Quaternion](https://ahrs.readthedocs.io/en/latest/quaternion/classQuaternion.html)
- Class [QuaternionArray](https://ahrs.readthedocs.io/en/lamy/quaternion/classQuaternionArray.html)
- The new class [Sensors](https://ahrs.readthedocs.io/en/latest/sensors.html) to simulate sensor data.
- The use of [Attitude estimation algorithms](https://ahrs.readthedocs.io/en/latest/filters.html).
- [Metrics functions](https://ahrs.readthedocs.io/en/latest/metrics.html) for orientation representations.
- The [World Magnetic Model](https://ahrs.readthedocs.io/en/latest/wmm.html)
- The [World Geodetic System](https://ahrs.readthedocs.io/en/latest/wgs84.html)
- And diverse tools included in `ahrs`.

### Helping Packages

Plotting and data-handling tools are imported from the script `tools.py` located in the current directory.

- `plot` shows time-series data in vertically stacked plots.
- `plot3` shows a 3D scene, where particles, frames, and items exist and interact in the same space.

Packages `matplotlib` and `ipympl` are required to build interactive visualizations in the Notebook. Make sure you have those installed.

These tools simplify the visualization of orientations in 3d, or time-series data, but are **NOT** included in the `ahrs` package.

Once you have `ahrs` installed (which also installs `numpy`) and you have the forementioned libraries, we can start by setting our notebook up.

In [ ]:
from madgwick_filter import compare, data_processing, utils

In [ ]:
# Use widgets
%matplotlib widget


# Import plotting tools
from madgwick_filter.tools_ahrs import plot
from madgwick_filter.tools_ahrs import plot3
import ahrs
import mrob
import twistnsync as tns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
plt.rcParams['font.size'] = 14
# Seed random generator
GENERATOR = np.random.default_rng(42)


## Attitude Estimators

Perhaps the most valued contribution of `ahrs` is its collection of attitude estimation algorithms. You can find a list [here](https://ahrs.readthedocs.io/en/lamy/filters.html)

Let's jut explore one famous example: The [Madgwick Filter](https://ahrs.readthedocs.io/en/lamy/filters/madgwick.html).


In [ ]:
x3_path = os.path.join("madgwick_filter", "TUM-VI", "imu0")
mocap_path = os.path.join("madgwick_filter", "TUM-VI", "mocap0")


In [ ]:
take_name = "room4.csv"

# Madgwick filter and game rotation vector comparison

In [ ]:
t_base, data_gyr, data_acc = data_processing.import_tum_imu(os.path.join(x3_path, take_name), smoothing=False)

In [ ]:
downscale = 1
t_base, data_gyr, data_acc = data_processing.downsample(downscale, t_base, data_gyr, data_acc)

In [ ]:
plot(data_gyr)

In [ ]:
freq_my = 120/downscale # Hz
# if frequency of MoCap is lower than resulting from data - change to MoCap's 240 Hz

Now that we generated IMU data, we can use it to estimate the original attitudes (orientations) with our Madgwick Filter.

In [ ]:
madgwick_IMU = ahrs.filters.Madgwick(gyr=data_gyr,
                                 acc=data_acc,
                                 frequency=freq_my)

Done!

The `Madgwick` object uses the given arrays to immediately perform the full computation of the orientations.

These orientations are in an $N\times 4$ array accessible in the attribute called `Q` (stands for Quaternions).

In [ ]:
plot(madgwick_IMU.Q)

Q = [w i j k] - [red green blue gold]

# Here goes comparison with Motion Capture as a reference

In [ ]:
data_mocap_t, data_mocap_q, data_mocap_T = data_processing.import_tum_mocap(os.path.join(mocap_path, take_name), smoothing=False)

In [ ]:
data_mocap_t, data_mocap_q, data_mocap_T = data_processing.downsample(downscale, data_mocap_t, data_mocap_q, data_mocap_T)

In [ ]:
plot(data_mocap_q, madgwick_IMU.Q)

In [ ]:
data_mocap_t - data_mocap_t[0]

In [ ]:
plot(data_mocap_t - data_mocap_t[0])

In [ ]:
t_base - t_base[0]

In [ ]:
#t_data_zeroed = (t_base - t_base[0]) / 1000
t_base, data_gyr, data_acc, data_acc, data_mocap_q = data_processing.sync_mocap_and_data(data_mocap_t, data_mocap_q, t_base, data_gyr, data_acc, data_acc)
# no more need in sync, because it's done after TwistnSync launch

In [ ]:
i_start = 0
t_base, data_gyr, data_acc, data_mocap_t, data_mocap_q = data_processing.arrays_from_i(i_start, t_base, data_gyr, data_acc, data_mocap_t, data_mocap_q)

In [ ]:
madgwick_shifted_IMU = ahrs.filters.Madgwick(gyr=data_gyr,
                                 acc=data_acc,
                                 frequency=freq_my)

In [ ]:
plot(madgwick_shifted_IMU.Q)

In [ ]:
plot(data_mocap_q, madgwick_shifted_IMU.Q)

In [ ]:
quat_sm_aligned = madgwick_shifted_IMU.Q
quat_mocap_aligned = data_mocap_q

In [ ]:
compare.errors_estimation(quat_sm_aligned, quat_mocap_aligned, source1="sensors")

# Sensors and Mocap data sync

More information can be found in the [paper](https://www.mdpi.com/1424-8220/21/1/68).

Here we find time offset and relative transformation between TUM-VI sensors output and Motion Capture system data of tracking

## Filter without magnetometer

In [ ]:
time_sync_imu, qsa, qra = compare.compare_smartphone_to_mocap(t_base, madgwick_shifted_IMU.Q, data_gyr,
                                                          data_mocap_t, data_mocap_q, 100, source1="sensors")

# Different plots

So this growing error (both APEs of R and g) in compare_to_mocap is just an integration (angvels -> quats) error. Maybe in my data it's also integration error?

How to mitigate? We can't rotate acc vector, tried

Maybe we can force sync quats when RPE is 0 ( -> now undistorted estimation)??